In [147]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import os

In [148]:
def get_query(list):
    # list generated from parser need to be formatted into http://www.espn.com/search/results?q=lonzo%20ball
    #                                                     www.espn.com/search/results?q=carl%20anthony%20towns
    search = 'http://www.espn.com/search/results?q='
    for each in list:
        search += each+'%20'
    search = search[:-3]
    return search

In [149]:
def get_url(query):
    driver.get(query)
    src = driver.page_source
    pretty_src = BeautifulSoup(src,'lxml')
    section_we_need = pretty_src.find_all(class_ = 'gsc-table-result')

    # after we have the section, extract all urls inside
    urls = []
    for each in section_we_need:
        each = str(each)
        url = re.findall('href=\"(.*?)\"', each)
        for every in url:
            urls.append(every)
            
    #uniqify
    uni_urls = []
    for each in urls:
        if each not in uni_urls:
            uni_urls.append(each)
    return uni_urls

In [150]:
# because we can only deal with stories right now:)
def get_story(urls):
    story = []
    for a in urls:
        if a.find('story')!=-1 and a.find('feature')==-1:
            story.append(a)
    return story

In [151]:
def get_header_article_per_url(url):
    driver.get(url)
    src = driver.page_source             # source html
    pretty_src = BeautifulSoup(src,'lxml')
    #print(pretty_src.prettify())
    # now we have the page, select all text from source    
    b = pretty_src.find('header', class_='article-header')
    header = re.sub('<.*?>','', str(b.h1))
    
    a = pretty_src.find('div', class_='article-body')  # all div with class="article-body"
    if len(a.find_all('p'))!=0:
        temp = a.find_all('p')
        temp = list(temp)
        temp = [str(i) for i in temp]
        content = "".join(temp)
        article = re.sub('<.*?>', '',content)
    return header, article

In [152]:
def retrieve_articles(urls):
    headers = []
    articles = []
    for each in urls:
        header, article = get_header_article_per_url(each)
        headers.append(header)
        articles.append(article)
    
    return headers, articles

In [158]:
def game_result(team, date):
    page = 'https://www.scoreboard.com/en/nba/results/'
    driver.get(page)
    src = driver.page_source             # source html
    pretty_src = BeautifulSoup(src,"lxml")
    info = pretty_src.tbody
    games = info.find_all('tr')
    time = []
    home = []
    away = []
    score = []
    for each in games:
        if each.find('td', class_ = 'cell_ad') != None:         #time
            time.append(each.find('td', class_ = 'cell_ad').string)
        if each.find('td', class_ = 'cell_ab') != None:         #home team
            home.append(each.find('td', class_ = 'cell_ab').span.string)
        if each.find('td', class_ = 'cell_ac') != None:         #away team
            away.append(each.find('td', class_ = 'cell_ac').span.string)
        if each.find('td', class_ = 'cell_sa') != None:         #score
            score.append(each.find('td', class_ = 'cell_sa').string)
    # we have team and date to put in!
    for i in range(len(home)):
        if (team == home[i] or team == away[i]) and time[i].find(date) != -1:
            homescore = int(score[i].split(':')[0])
            awayscore = int(score[i].split(':')[1])
            if team == home[i]:
                if homescore > awayscore:
                    print('{} won with a score of: {}:{}!'.format(team,homescore,awayscore))
                else:
                    print('{} lost with a score of: {}:{}!'.format(team, awayscore, homescore))
            else:
                if homescore > awayscore:
                    print('{} lost with a score of: {}:{}!'.format(team,awayscore,homescore))           
                else:
                    print('{} won with a score of: {}:{}!'.format(team, homescore, awayscore))
            break

In [154]:
# set before working
abspath = os.path.abspath(r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")  
driver = webdriver.Chrome(abspath)

Presentation part

In [155]:
query = get_query(['donovan','mitchell'])
urls = get_url(query)
story = get_story(urls)
headers, articles = retrieve_articles(story[0:2])

In [159]:
# This cell is for getting results and stats of a team/player directly.
game_result(team = 'Toronto Raptors', date = '05.05')

Toronto Raptors lost with a score of: 103:105!


In [157]:
headers[0],articles[0]

("Donovan Mitchell: 'Terrible shots' led to poor play in loss to Rockets",
 'SALT LAKE CITY -- Donovan Mitchell\'s perspective on fellow Rookie of the Year candidate Ben Simmons\' struggles in the conference semifinals was unfortunately prophetic for the Utah Jazz star guard.Asked Friday morning about Simmons\' one-point, five-turnover outing in the Philadelphia 76ers\' Game 2 loss to the Boston Celtics, Mitchell answered in part, "It happens to everybody."The Houston Rockets bounced back from an uneven Game 2 performance by making the second half of Game 3 in Salt Lake City irrelevant.Hours later, Mitchell had by far his worst playoff performance as the Houston Rockets routed the Jazz 113-92 in Game 3 to regain the advantage in the Western Conference semifinals series.Mitchell missed 10 of his first 11 field goal attempts and finished with a personal playoff-low 10 points on 4-of-16 shooting. Mitchell, who is playing point guard with Ricky Rubio sidelined by a hamstring injury, had as